In [20]:
#import statements
import pandas as pd 
import numpy as np 
import tensorflow as tf 
from pathlib import Path
import os
import sys
from sklearn.model_selection import train_test_split
from helper_methods import print_schema, encode_numeric_zscore, encode_text_dummy, print_column, encode_text_index, to_xy, plot_losses
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn import metrics
from sklearn.metrics import roc_curve, auc, confusion_matrix
import matplotlib.pyplot as plt 

### Define Helper Methods

In [21]:
def plot_confusion_matrix(cm, names, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(names))
    plt.xticks(tick_marks, names, rotation=45)
    plt.yticks(tick_marks, names)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    

# Plot an ROC. pred - the predictions, y - the expected output.
def plot_roc(pred,y, file_path):
    fpr, tpr, thresholds = roc_curve(y, pred)
    roc_auc = auc(fpr, tpr)

    plt.figure()
    plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC)')
    plt.legend(loc="lower right")
    plt.savefig(file_path)
    plt.close()

### Setup Environment
Create folders for putting test output in

In [22]:
base_path=os.path.join(os.getcwd(), 'test-output/')

iteration='iteration-3'
full_path = os.path.join(base_path, iteration)
try:
        os.mkdir(base_path)
except Exception as e:
     print(f"Probably safe to ignore the following error: \n{e}")
try:
    os.mkdir(full_path)
except Exception as e:
    print(f"{e}\nExiting to protect previous work.")
    sys.exit(0)
    

Probably safe to ignore the following error: 
[WinError 183] Cannot create a file when that file already exists: 'c:\\Users\\timef\\Documents\\Workspaces\\Python\\csc180\\network-traffic-analyzer\\test-output/'


In [23]:
#read from train/test datasets
training_dataframe = pd.read_csv(os.path.join(os.getcwd(), 'data/UNSW_NB15_training-set.csv'))
testing_dataframe = pd.read_csv(os.path.join(os.getcwd(), 'data/UNSW_NB15_testing-set.csv'))

In [24]:
#drop all entries that have a missing value anywhere in the set
training_dataframe = training_dataframe.replace('-', np.nan).dropna()
testing_dataframe = testing_dataframe.replace('-', np.nan).dropna()


In [25]:
testing_dataframe.shape

(35179, 45)

In [26]:
#get one instance of each value from each dataframe
training_unique = training_dataframe["proto"].unique()
testing_unique = testing_dataframe["proto"].unique()

#get only values from each numpy array
common_values = np.intersect1d(training_unique, testing_unique)

#make a filtered dataframe containing only values in common_values
training_dataframe = training_dataframe[training_dataframe['proto'].isin(common_values)]
testing_dataframe = testing_dataframe[testing_dataframe['proto'].isin(common_values)]

#same for service
training_unique = training_dataframe["service"].unique()
testing_unique = testing_dataframe["service"].unique()
common_values = np.intersect1d(training_unique, testing_unique)
training_dataframe = training_dataframe[training_dataframe['service'].isin(common_values)]
testing_dataframe = testing_dataframe[testing_dataframe['service'].isin(common_values)]

#same for state
training_unique = training_dataframe["state"].unique()
testing_unique = testing_dataframe["state"].unique()
common_values = np.intersect1d(training_unique, testing_unique)
training_dataframe = training_dataframe[training_dataframe['state'].isin(common_values)]
testing_dataframe = testing_dataframe[testing_dataframe['state'].isin(common_values)]


In [27]:
testing_dataframe.shape

(35178, 45)

In [28]:
print_schema(training_dataframe)
print_schema(testing_dataframe)


~~~~~~dataframe schema~~~~~~
Dataframe shape: (81159, 45) | Dataframe length: 81159
Column labels: 
Index(['id', 'dur', 'proto', 'service', 'state', 'spkts', 'dpkts', 'sbytes',
       'dbytes', 'rate', 'sttl', 'dttl', 'sload', 'dload', 'sloss', 'dloss',
       'sinpkt', 'dinpkt', 'sjit', 'djit', 'swin', 'stcpb', 'dtcpb', 'dwin',
       'tcprtt', 'synack', 'ackdat', 'smean', 'dmean', 'trans_depth',
       'response_body_len', 'ct_srv_src', 'ct_state_ttl', 'ct_dst_ltm',
       'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm',
       'is_ftp_login', 'ct_ftp_cmd', 'ct_flw_http_mthd', 'ct_src_ltm',
       'ct_srv_dst', 'is_sm_ips_ports', 'attack_cat', 'label'],
      dtype='object')
Dataframe head: 
    id       dur proto service state  spkts  dpkts  sbytes  dbytes  \
3    4  1.681642   tcp     ftp   FIN     12     12     628     770   
11  12  2.093085   tcp    smtp   FIN     62     28   56329    2212   
15  16  0.000002   udp    snmp   INT      2      0     138       0   
17  18  

### Encode Data

One-hot encode relevant rows


In [29]:
# one-hot encode non-y features of training dataset
encode_text_dummy(training_dataframe, 'proto')
encode_text_dummy(training_dataframe, 'service')
encode_text_dummy(training_dataframe, 'state')

# one-hot encode attack-category column (y)

# encode_text_index(training_dataframe, 'attack_cat')
# one-hot encode non-y features of testing dataset
encode_text_dummy(testing_dataframe, 'proto')
encode_text_dummy(testing_dataframe, 'service')
encode_text_dummy(testing_dataframe, 'state')
# create a copy of the dataset for use in the MCC CNN
mcc_testingdataframe = testing_dataframe.copy(deep=True)
mcc_trainingdataframe = training_dataframe.copy(deep=True)
mcc_testingdataframe.drop('label', inplace=True, axis=1)
mcc_trainingdataframe.drop('label' , inplace=True, axis=1)


training_dataframe.drop('attack_cat', inplace=True, axis=1)
# one-hot encode attack-category column (y)
testing_dataframe.drop('attack_cat', inplace=True, axis=1)
# encode_text_index(testing_dataframe, 'attack_cat')

In [30]:
print_schema(training_dataframe)
print_schema(testing_dataframe)


~~~~~~dataframe schema~~~~~~
Dataframe shape: (81159, 59) | Dataframe length: 81159
Column labels: 
Index(['id', 'dur', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'sttl',
       'dttl', 'sload', 'dload', 'sloss', 'dloss', 'sinpkt', 'dinpkt', 'sjit',
       'djit', 'swin', 'stcpb', 'dtcpb', 'dwin', 'tcprtt', 'synack', 'ackdat',
       'smean', 'dmean', 'trans_depth', 'response_body_len', 'ct_srv_src',
       'ct_state_ttl', 'ct_dst_ltm', 'ct_src_dport_ltm', 'ct_dst_sport_ltm',
       'ct_dst_src_ltm', 'is_ftp_login', 'ct_ftp_cmd', 'ct_flw_http_mthd',
       'ct_src_ltm', 'ct_srv_dst', 'is_sm_ips_ports', 'label', 'proto-tcp',
       'proto-udp', 'service-dhcp', 'service-dns', 'service-ftp',
       'service-ftp-data', 'service-http', 'service-irc', 'service-pop3',
       'service-radius', 'service-smtp', 'service-snmp', 'service-ssh',
       'service-ssl', 'state-CON', 'state-FIN', 'state-INT', 'state-REQ'],
      dtype='object')
Dataframe head: 
    id       dur  spkts  dpkts  sbytes 

Z-Score relevant rows

In [31]:
for column in training_dataframe.columns:
    if column not in ['service', 'srcip', 'dstip', 'proto', 'state','service', 'attack_cat', 'label', 'is_ftp_login', 'is_sm_ips_ports', 'Stime', 'Ltime']:
        encode_numeric_zscore(training_dataframe, column)
for column in testing_dataframe.columns:
    if column not in ['service', 'srcip', 'dstip', 'proto', 'state','service', 'attack_cat', 'label', 'is_ftp_login', 'is_sm_ips_ports', 'Stime', 'Ltime']:
        encode_numeric_zscore(testing_dataframe, column)
for column in mcc_trainingdataframe.columns:
    if column not in ['service', 'srcip', 'dstip', 'proto', 'state','service', 'attack_cat', 'label', 'is_ftp_login', 'is_sm_ips_ports', 'Stime', 'Ltime']:
        encode_numeric_zscore(mcc_trainingdataframe, column)
for column in mcc_testingdataframe.columns:
    if column not in ['service', 'srcip', 'dstip', 'proto', 'state','service', 'attack_cat', 'label', 'is_ftp_login', 'is_sm_ips_ports', 'Stime', 'Ltime']:
        encode_numeric_zscore(mcc_trainingdataframe, column)

In [32]:
print_schema(training_dataframe)
print_schema(testing_dataframe)

~~~~~~dataframe schema~~~~~~
Dataframe shape: (81159, 59) | Dataframe length: 81159
Column labels: 
Index(['id', 'dur', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'sttl',
       'dttl', 'sload', 'dload', 'sloss', 'dloss', 'sinpkt', 'dinpkt', 'sjit',
       'djit', 'swin', 'stcpb', 'dtcpb', 'dwin', 'tcprtt', 'synack', 'ackdat',
       'smean', 'dmean', 'trans_depth', 'response_body_len', 'ct_srv_src',
       'ct_state_ttl', 'ct_dst_ltm', 'ct_src_dport_ltm', 'ct_dst_sport_ltm',
       'ct_dst_src_ltm', 'is_ftp_login', 'ct_ftp_cmd', 'ct_flw_http_mthd',
       'ct_src_ltm', 'ct_srv_dst', 'is_sm_ips_ports', 'label', 'proto-tcp',
       'proto-udp', 'service-dhcp', 'service-dns', 'service-ftp',
       'service-ftp-data', 'service-http', 'service-irc', 'service-pop3',
       'service-radius', 'service-smtp', 'service-snmp', 'service-ssh',
       'service-ssl', 'state-CON', 'state-FIN', 'state-INT', 'state-REQ'],
      dtype='object')
Dataframe head: 
          id       dur     spkts     dp

In [33]:
x_train,y_train=to_xy(training_dataframe, 'label')
x_test, y_test=to_xy(testing_dataframe, 'label')

### Create Fully Connected Model
Design the model's architecture. 

In [34]:
# TODO implement a more robust training setup, implement model tabulation 

checkpointer = ModelCheckpoint(filepath=os.path.join(full_path, "fcn-best_weights.keras"), verbose=0, save_best_only=True) # save best model
for i in range(5):
        print(f"training FCN: {i}")
        model = Sequential()
        model.add(Dense(64, input_dim=x_train.shape[1], activation='relu'))
        model.add(Dropout(0.3))
        model.add(Dense(32, activation='relu'))
        model.add(Dropout(0.3))
        model.add(Dense(16, activation='relu'))
        model.add(Dropout(0.3))
        model.add(Dense(8, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(2, activation='sigmoid')) # single output w/o activation function b/c regression
        model.compile(loss='binary_crossentropy', optimizer=SGD(learning_rate=0.001, momentum=0.01))
        monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=10, verbose=2, mode='min', restore_best_weights=True)
        history = model.fit(x_train, y_train, validation_data=(x_test, y_test),batch_size=64, callbacks=[monitor, checkpointer], verbose=1, epochs=100)
        plot_losses(history, full_path, i)

training FCN: 0
Epoch 1/100


c:\Users\timef\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1269/1269 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.5109 - val_loss: 0.3944
Epoch 2/100
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.3867 - val_loss: 0.3646
Epoch 3/100
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.3385 - val_loss: 0.3652
Epoch 4/100
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.3107 - val_loss: 0.3689
Epoch 5/100
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.2844 - val_loss: 0.3683
Epoch 6/100
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.2632 - val_loss: 0.3647
Epoch 7/100
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.2436 - val_loss: 0.3588
Epoch 8/100
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.2284 - val_loss: 0.3440
Epoch 9/100
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.2075 - val_loss: 0.3376
Epoch 10/100
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.1924 - val_loss: 0.3310
Epoch 11/100
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.1797 - val_loss: 0.3219
Epoch 12/100
1269/1269 ━━━━━━━━

In [35]:
# load model, do prediction, do evaulation
model.load_weights(os.path.join(full_path, "fcn-best_weights.keras"))
prediction = model.predict(x_test)
pred_roc = prediction[:,1]
y_true = np.argmax(y_test, axis=1)

plot_roc(pred_roc, y_true, os.path.join(full_path, 'fcn-roc-curve.png'))
try:
        pred = np.argmax(prediction, axis=1)
        # confusion matrix
        cm = confusion_matrix(y_true, pred)
        plt.figure()
        plot_confusion_matrix(cm, ['normal', 'malicious'])
        plt.savefig(os.path.join(full_path, 'fcn-confusion-matrix.png'))
        plt.close()
except Exception as e:
        print(f"error creating confusion matrix plot:\n{e} ")
try:
        with open(os.path.join(full_path, 'fcn-metrics.txt'), 'x') as file:
                prediction=np.argmax(prediction, axis=1)
                y_true = np.argmax(y_test, axis=1)
                accuracy_score = metrics.accuracy_score(y_true, prediction)
                precision_score = metrics.precision_score(y_true, prediction, average='weighted')
                recall_score = metrics.recall_score(y_true, prediction, average= "weighted")
                fl_score = metrics.f1_score(y_true, prediction, average= "weighted")
                file.write(f"Accuracy Score: {accuracy_score}\n")
                file.write(f"Precision Score: {precision_score}\n")
                file.write("Recall score: {}\n".format(recall_score))
                file.write("F1 score: {}\n".format(fl_score))
                log_loss = metrics.log_loss(y_test, prediction)
                print(f"Log Loss: {log_loss}")
                file.write(str(metrics.classification_report(y_true, prediction)))
                file.write("\nNumpy array of predictions\n")
                file.write(np.array_str(prediction[0:5]))
                file.write("y_test:\n")
                file.write(np.array2string(y_test[0:5]))
except OSError as e:
        print(f"Error while writing model metrics: \n{e}")

1100/1100 ━━━━━━━━━━━━━━━━━━━━ 1s 595us/step
Log Loss: 3.020543811220574


### Connected Neural Network
Design the CNN

In [36]:
#CNN requires 3D input (already float32)
x_train_cnn = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
x_test_cnn = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)

In [37]:
checkpointer = ModelCheckpoint(filepath=os.path.join(full_path, "cnn-best_weights.keras"), verbose=0, save_best_only=True) # save best model

#build CNN
model = Sequential()

#convolution blocks
model.add(Conv1D(64, kernel_size=3, activation='relu', padding='valid', input_shape=(x_train_cnn.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))
model.add(Conv1D(32, kernel_size=3, strides=1, padding='valid', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))
model.add(Conv1D(16, kernel_size=3, strides=1, padding='valid', activation='relu'))
model.add(Dropout(0.2))

#flatten and dense layers
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2, activation='sigmoid'))

#compilation
model.compile(loss='binary_crossentropy', optimizer=SGD(learning_rate=0.001, momentum=0.01),)
#early stopping
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=10, verbose=2, mode='min', restore_best_weights=True)
#fit the model
history = model.fit(x_train_cnn, y_train, validation_data=(x_test_cnn, y_test), batch_size=64, callbacks=[monitor, checkpointer], verbose=1, epochs=100)




Epoch 1/100


c:\Users\timef\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1269/1269 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 0.6129 - val_loss: 0.5799
Epoch 2/100
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.5426 - val_loss: 0.5337
Epoch 3/100
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.4847 - val_loss: 0.4713
Epoch 4/100
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.4278 - val_loss: 0.4240
Epoch 5/100
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.3834 - val_loss: 0.3988
Epoch 6/100
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.3525 - val_loss: 0.3804
Epoch 7/100
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.3253 - val_loss: 0.3638
Epoch 8/100
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.2997 - val_loss: 0.3547
Epoch 9/100
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.2763 - val_loss: 0.3476
Epoch 10/100
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.2560 - val_loss: 0.3403
Epoch 11/100
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.2366 - val_loss: 0.3426
Epoch 12/100
1269/1269 ━━━━━━━━

In [38]:
# load model, do prediction, do evaulation
model.load_weights(os.path.join(full_path, "cnn-best_weights.keras"))
prediction = model.predict(x_test)
pred_roc = prediction[:,1]
y_true = np.argmax(y_test, axis=1)

plot_roc(pred_roc, y_true, os.path.join(full_path, 'cnn-roc-curve.png'))
try:
        pred = np.argmax(prediction, axis=1)
        # confusion matrix
        cm = confusion_matrix(y_true, pred)
        plt.figure()
        plot_confusion_matrix(cm, ['normal', 'malicious'])
        plt.savefig(os.path.join(full_path, 'cnn-confusion-matrix.png'))
        plt.close()
except Exception as e:
        print(f"error creating confusion matrix plot:\n{e} ")
try:
        with open(os.path.join(full_path, 'cnn-metrics.txt'), 'x') as file:
                prediction=np.argmax(prediction, axis=1)
                y_true = np.argmax(y_test, axis=1)
                accuracy_score = metrics.accuracy_score(y_true, prediction)
                precision_score = metrics.precision_score(y_true, prediction, average='weighted')
                recall_score = metrics.recall_score(y_true, prediction, average= "weighted")
                fl_score = metrics.f1_score(y_true, prediction, average= "weighted")
                file.write(f"Accuracy Score: {accuracy_score}\n")
                file.write(f"Precision Score: {precision_score}\n")
                file.write("Recall score: {}\n".format(recall_score))
                file.write("F1 score: {}\n".format(fl_score))
                log_loss = metrics.log_loss(y_test, prediction)
                print(f"Log Loss: {log_loss}")                
                file.write(str(metrics.classification_report(y_true, prediction)))
                file.write("\nNumpy array of predictions\n")
                file.write(np.array_str(prediction[0:5]))
                file.write("y_test:\n")
                file.write(np.array2string(y_test[0:5]))
except OSError as e:
        print(f"Error while writing model metrics: \n{e}")

1100/1100 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
Log Loss: 2.897590874535883


### Multi-Class Classifier

Morph the dataset for use in a Multi-class classifier CNN

In [39]:
attack_categories = encode_text_index(mcc_trainingdataframe, 'attack_cat')
encode_text_index(mcc_testingdataframe, 'attack_cat')
mcc_x_train, mcc_y_train = to_xy(mcc_trainingdataframe, 'attack_cat')
mcc_x_test, mcc_y_test = to_xy(mcc_trainingdataframe, 'attack_cat')
print(f"{mcc_x_test.shape}, {mcc_y_train.shape}, {mcc_x_test.shape}, {mcc_y_test.shape}")

(81159, 58), (81159, 9), (81159, 58), (81159, 9)


In [40]:
x_train_mcc_cnn = mcc_x_train.reshape(mcc_x_train.shape[0], mcc_x_train.shape[1], 1)
x_test_mcc_cnn = mcc_x_test.reshape(mcc_x_test.shape[0], mcc_x_test.shape[1], 1)

In [41]:
print(f"{x_train_mcc_cnn.shape}, {x_test_mcc_cnn.shape}")

(81159, 58, 1), (81159, 58, 1)


#### Define Model

In [44]:
checkpointer = ModelCheckpoint(filepath=os.path.join(full_path, "mcc_cnn-best_weights.keras"), verbose=0, save_best_only=True) # save best model
for i in range(5):
    #build CNN
    model = Sequential()

    #convolution blocks
    model.add(Conv1D(64, kernel_size=3, activation='relu', padding='valid', input_shape=(x_train_cnn.shape[1], 1)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.3))
    model.add(Conv1D(32, kernel_size=3, strides=1, padding='valid', activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.3))
    model.add(Conv1D(16, kernel_size=3, strides=1, padding='valid', activation='relu'))
    model.add(Dropout(0.2))

    #flatten and dense layers
    model.add(Flatten())
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(16, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(y_train.shape[1], activation='softmax'))

    #compilation
    model.compile(loss='categorical_crossentropy', optimizer=SGD(learning_rate=0.01, momentum=0.01),)
    #early stopping
    monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=10, verbose=2, mode='min', restore_best_weights=True)
    #fit the model
    history = model.fit(x_train_cnn, y_train, validation_data=(x_test_cnn, y_test), batch_size=64, callbacks=[monitor, checkpointer], verbose=1, epochs=100)
    plot_losses(history, full_path, i)

Epoch 1/100


c:\Users\timef\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1269/1269 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.3198 - val_loss: 0.2773
Epoch 2/100
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.1510 - val_loss: 0.2460
Epoch 3/100
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.1170 - val_loss: 0.2346
Epoch 4/100
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.1031 - val_loss: 0.2091
Epoch 5/100
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.0953 - val_loss: 0.2218
Epoch 6/100
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.0916 - val_loss: 0.2217
Epoch 7/100
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.0860 - val_loss: 0.2321
Epoch 8/100
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.0828 - val_loss: 0.2210
Epoch 9/100
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.0803 - val_loss: 0.2319
Epoch 10/100
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.0785 - val_loss: 0.2345
Epoch 11/100
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.0773 - val_loss: 0.2308
Epoch 12/100
1269/1269 ━━━━━━━━

#### Save Metrics, Architecture, Plots


In [47]:
# load model, do prediction, do evaulation
model.load_weights(os.path.join(full_path, "mcc_cnn-best_weights.keras"))
prediction = model.predict(x_test)
pred_roc = prediction[:,1]
y_true = np.argmax(y_test, axis=1)

plot_roc(pred_roc, y_true, os.path.join(full_path, 'mcc_cnn-roc-curve.png'))
try:
        pred = np.argmax(prediction, axis=1)
        # confusion matrix
        cm = confusion_matrix(y_true, pred)
        plt.figure()
        plot_confusion_matrix(cm, attack_categories)
        plt.savefig(os.path.join(full_path, 'mcc_cnn-confusion-matrix.png'))
        plt.close()
except Exception as e:
        print(f"error creating confusion matrix plot:\n{e} ")
try:
        with open(os.path.join(full_path, 'mcc_cnn-metrics.txt'), 'x') as file:
                prediction=np.argmax(prediction, axis=1)
                y_true = np.argmax(y_test, axis=1)
                accuracy_score = metrics.accuracy_score(y_true, prediction)
                precision_score = metrics.precision_score(y_true, prediction, average='weighted')
                recall_score = metrics.recall_score(y_true, prediction, average= "weighted")
                fl_score = metrics.f1_score(y_true, prediction, average= "weighted")
                file.write(f"Accuracy Score: {accuracy_score}\n")
                file.write(f"Precision Score: {precision_score}\n")
                file.write("Recall score: {}\n".format(recall_score))
                file.write("F1 score: {}\n".format(fl_score))
                log_loss = metrics.log_loss(y_test, prediction)
                file.write(f"Log Loss: {log_loss}")                
                file.write(str(metrics.classification_report(y_true, prediction)))
                file.write("\nNumpy array of predictions\n")
                file.write(np.array_str(prediction[0:5]))
                file.write("y_test:\n")
                file.write(np.array2string(y_test[0:5]))
except OSError as e:
        print(f"Error while writing model metrics: \n{e}")

1100/1100 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
